In [1]:
import sklearn

#NN Surrogate model class
from injector_surrogate_quads import *

from emittance_calc import getNormEmit

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#BO
from bayes_opt import BayesianOptimization

# Load injector model

In [2]:
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-19T09_46_52-07_00')
Model.load_scaling()
Model.take_log_out = False

energy = 0.204 #GeV

# Set up sampling and objectives

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

1- beam size request (from surrogate) [input =  setting, quad (n=1), output = beamsize] ----> uncouple from fn below \
2- emittance calc request [input = quad (n>=3), beamsize (n), output  = emittance] ----> fn from module\
3- ground truth request [input = setting, output = emittance] -----> already exists, need wrapper fn 

In [5]:
#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient','QE04:b1_gradient']

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def getBeamsize(varx,vary,varz,varscan): 
    
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz
    
    #set quad 525 to values for scan
    x_in[:, Model.loc_in[opt_var_names[3]]] = varscan

    #output predictions
    y_out = Model.pred_machine_units(x_in) 
    
    x_rms = y_out[:,0]
    y_rms = y_out[:,1]

    return x_rms, y_rms

   ####################

    
normemit = getNormEmit(energy, quad_grad, x_rms, y_rms) #normalized emittance
# getNormEmit if can't calc emittance, assigns random high val (1000)

def getGroundTruth(varx,vary,varz): 
    
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz

    #output predictions
    y_out = Model.pred_machine_units(x_in) 
    
    #output is geometric emittance in transverse plane
    emitx = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    emity = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    
    return np.sqrt(emitx*emity)

# Simple BO

In [8]:
# bounds on input params 
pbounds = {'varx': (0.44, 0.55),
           'vary': (-0.02, 0.02),
           'varz': (-0.02, 0.02),
           #'varscan': (-7, 0), # how to pick set of quads when scanning?
          }

optimizer = BayesianOptimization(
    f = evaluate,
    pbounds = pbounds,
    random_state = 1,
)

optimizer.maximize(
    init_points=5,
    n_iter=40,
)

|   iter    |  target   |   varx    |   vary    |   varz    |
-------------------------------------------------------------
|  1        | -6.656    |  0.4859   |  0.008813 | -0.02     |
|  2        | -5.68     |  0.4733   | -0.01413  | -0.01631  |
|  3        | -13.23    |  0.4605   | -0.006178 | -0.004129 |
|  4        | -7.805    |  0.4993   | -0.003232 |  0.007409 |
|  5        | -16.73    |  0.4625   |  0.01512  | -0.0189   |
|  6        | -4.989    |  0.4813   |  0.0116   |  0.01098  |
|  7        | -16.46    |  0.4986   |  0.0115   | -0.013    |
|  8        | -5.931    |  0.5055   | -0.009559 |  0.009869 |
|  9        | -9.944    |  0.465    | -0.01378  |  0.01789  |
|  10       | -5.85     |  0.4732   | -0.0146   | -0.01722  |
|  11       | -7.375    |  0.4917   | -0.00306  | -0.001123 |
|  12       | -4.571    |  0.4823   | -0.005622 | -0.01817  |
|  13       | -7.577    |  0.4995   | -0.003835 |  0.007813 |
|  14       | -5.944    |  0.5055   | -0.01029  |  0.009299 |
|  15   

## Results from simple BO

In [6]:
SOL_opt = optimizer.max['params']['varx'] # solenoid val at optimum
CQ_opt = optimizer.max['params']['vary'] # CQ val at optimum
SQ_opt = optimizer.max['params']['varz'] # SQ val at optimum

opt_emit = -1*optimizer.max['target'] # emittance value at optimum (in um)

print('optimum (pv_units) ',SOL_opt, CQ_opt, SQ_opt)
print('optimum geom emit ', opt_emit)

optimum (pv_units)  0.4783371750558134 -0.02 0.02
optimum geom emit  0.8672671914100647
